In [7]:
import os
import glob
import pandas as pd
import numpy as np

In [3]:
dir_p = '/SNOWDATA/uavsar/imgs'

In [16]:
os.path.join(dir_p, '*/*'),

('/SNOWDATA/uavsar/imgs/*/*',)

In [41]:
res = pd.DataFrame()
for i, fp in enumerate(glob.glob(os.path.join(dir_p, '*/*.csv'), recursive=True)):
    df = pd.read_csv(fp)
    sub = df[['start time of acquisition for pass 1','stop time of acquisition for pass 1','start time of acquisition for pass 2','stop time of acquisition for pass 2','peg heading','approximate upper left latitude',
'approximate upper left longitude','approximate upper right latitude','approximate upper right longitude','approximate lower left latitude','approximate lower left longitude','approximate lower right latitude','approximate lower right longitude']]
    res = res.append(sub.iloc[0], ignore_index=True)

In [43]:
res.to_csv('../../figures/imgs_map_meta.csv')